## Creating the AWS Serverless Application Model (SAM) application

### Verify pre-requisites for AWS SAM
SAM requires python 2.7 or 3.6 and pip to be installed to continue.

In [64]:
import os
import sagemaker
from sagemaker import get_execution_role
# import project_path
# from lib import utils
import boto3

role = get_execution_role()

# use the region-specific sample data bucket
region = boto3.Session().region_name
bucket = '{{s3_workshop_bucket}}'
stack_name = "serverless-hello"

In [ ]:
!python --version
!pip --version

### Installing AWS SAM

In [ ]:
!pip install --user aws-sam-cli

### Verify the latest version of SAM is installed 

In [2]:
!python -m site --user-base

/home/ec2-user/.local


In [87]:
%%bash
export PATH=$PATH:/home/ec2-user/.local/bin
sam --version

SAM CLI, version 0.10.0


In [26]:
%bash
/home/ec2-user/.local/bin/sam init -r python3.6 -n serverless-todo

[+] Initializing project structure...
[SUCCESS] - Read serverless-todo/README.md for further instructions on how to proceed
[*] Project initialization is now complete


In [27]:
!ls serverless-todo/

hello_world  README.md	template.yaml  tests


In [29]:
!cat serverless-todo/template.yaml

AWSTemplateFormatVersion: '2010-09-09'
Transform: AWS::Serverless-2016-10-31
Description: >
    serverless-todo

    Sample SAM Template for serverless-todo

# More info about Globals: https://github.com/awslabs/serverless-application-model/blob/master/docs/globals.rst
Globals:
    Function:
        Timeout: 3


Resources:

    HelloWorldFunction:
        Type: AWS::Serverless::Function # More info about Function Resource: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#awsserverlessfunction
        Properties:
            CodeUri: hello_world/
            Handler: app.lambda_handler
            Runtime: python3.6
            Environment: # More info about Env Vars: https://github.com/awslabs/serverless-application-model/blob/master/versions/2016-10-31.md#environment-object
                Variables:
                    PARAM1: VALUE
            Events:
                HelloWorld:
                    Type: Api # More info about API Event Sourc

In [ ]:
cd serverless-todo 

In [85]:
!pip install -r hello_world/requirements.txt -t hello_world/

  Using cached https://files.pythonhosted.org/packages/f1/ca/10332a30cb25b627192b4ea272c351bce3ca1091e541245cccbace6051d8/requests-2.20.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/e0/accfc1b56b57e9750eba272e24c4dddeac86852c2bebd1236674d7887e8a/certifi-2018.11.29-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/00/ee1d7de624db8ba7090d1226aebefab96a2c71cd5cfa7629d6ad3f61b79e/urllib3-1.24.1-py2.py3-none-any.whl
aws-sam-cli 0.10.0 has requirement requests==2.20.1, but you'll have requests 2.20.0 which is incompatible.
Target directory /home/ec2-user/SageMaker/aws-research-workshops/notebooks/serverless_apps/serverless-to

In [80]:
!ls hello_world

app.py			      __init__.py
bin			      __pycache__
certifi			      requests
certifi-2018.11.29.dist-info  requests-2.20.0.dist-info
chardet			      requirements.txt
chardet-3.0.4.dist-info       urllib3
idna			      urllib3-1.24.1.dist-info
idna-2.7.dist-info


In [53]:
!pygmentize hello_world/app.py

import json

import requests


def lambda_handler(event, context):
    """Sample pure Lambda function

    Parameters
    ----------
    event: dict, required
        API Gateway Lambda Proxy Input Format

        {
            "resource": "Resource path",
            "path": "Path parameter",
            "httpMethod": "Incoming request's method name"
            "headers": {Incoming request headers}
            "queryStringParameters": {query string parameters }
            "pathParameters":  {path parameters}
            "stageVariables": {Applicable stage variables}
            "requestContext": {Request context, including authorizer-returned key-value pairs}
            "body": "A JSON string of the request payload."
            "isBase64Encoded": "A boolean flag to indicate if the applicable request payload is Base64-encode"
        }

        https://docs.aws.amazon.com/apigateway/latest/developerguide/set-up-lambda-proxy-integrations.html#api-gateway-simple-proxy-for-lambda-inpu

In [81]:
!aws cloudformation validate-template --template-body file://template.yaml

{
    "Parameters": [],
    "Description": "serverless-todo\nSample SAM Template for serverless-todo\n"
}


In [82]:
!aws cloudformation package \
    --template-file template.yaml \
    --output-template-file sam-template.yaml \
    --s3-bucket analytics-serverless-west \
    --s3-prefix serverless

Uploading to serverless/8c12943ceafd7411b2d60bf9e4a10e6d  965088 / 965088.0  (100.00%)
Successfully packaged artifacts and wrote output template to file sam-template.yaml.
Execute the following command to deploy the packaged template
aws cloudformation deploy --template-file /home/ec2-user/SageMaker/aws-research-workshops/notebooks/serverless_apps/serverless-todo/sam-template.yaml --stack-name <YOUR STACK NAME>


In [83]:
!aws cloudformation deploy \
    --template-file sam-template.yaml \
    --stack-name serverless-hello \
    --capabilities CAPABILITY_IAM


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - serverless-hello


In [71]:
# api_endpoint = !aws cloudformation describe-stacks --stack-name $stack_name --region $region | python -c 'import json,sys;obj=json.load(sys.stdin);print obj["Stacks"][0]["Outputs"][1]["OutputValue"];'
# print(api_endpoint[0])

!aws cloudformation describe-stacks --stack-name $stack_name --region $region

{
    "Stacks": [
        {
            "StackId": "arn:aws:cloudformation:us-west-2:649037252677:stack/serverless-hello/1bc40450-0cac-11e9-9a6e-028572da108e",
            "StackName": "serverless-hello",
            "ChangeSetId": "arn:aws:cloudformation:us-west-2:649037252677:changeSet/awscli-cloudformation-package-deploy-1546227907/9f6a077a-b6ec-49e2-a1b3-54e25b13deae",
            "Description": "serverless-todo\nSample SAM Template for serverless-todo\n",
            "CreationTime": "2018-12-31T03:28:15.142Z",
            "LastUpdatedTime": "2018-12-31T03:45:13.188Z",
            "RollbackConfiguration": {},
            "StackStatus": "UPDATE_COMPLETE",
            "DisableRollback": false,
            "NotificationARNs": [],
            "Capabilities": [
                "CAPABILITY_IAM"
            ],
            "Outputs": [
                {
                    "OutputKey": "HelloWorldFunctionIamRole",
                    "OutputValue": "arn:aws:iam::649037252677:role/serverles

After we create and deploy the project we will install httpie to test the endpoint to exercise the default method.

In [84]:
!pip install httpie
!http https://ud6tcavio8.execute-api.us-west-2.amazonaws.com/Prod/hello/

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 54
Content-Type: application/json
Date: Mon, 31 Dec 2018 03:56:25 GMT
Via: 1.1 5ecfd97124a5b15f1245ab731d34efe7.cloudfront.net (CloudFront)
X-Amz-Cf-Id: yt-uhIb01LanCQjCH6hzvDlMtW70ELSVBrru-gNc0Ih089MO8HwhTA==
X-Amzn-Trace-Id: Root=1-5c299368-804f0e98fa9f6cc0436d5a30;Sampled=0
X-Cache: Miss from cloudfront
x-amz-apigw-id: SwP4YGpyPHcF7Fw=
x-amzn-RequestId: 0af20009-0cb0-11e9-a381-af13594a8ac7

{
    "location": "54.202.67.39",
    "message": "hello world"
}



### We will now make changes to the hello world project and create a todo application that leverages Cognito for user authentication.

We will start with creating the CRUD methods for the todo app to create, read, update, delete, and list the todos. We will also be using a DynamoDB table to store the todos. 

In your project copy the todos folder over to your project and update the serverless.yml file with the new API endpoints.

```
create:
    handler: todos/create.create
    events:
      - http:
          path: todos
          method: post
          cors: true

  list:
    handler: todos/list.list
    events:
      - http:
          path: todos
          method: get
          cors: true

  get:
    handler: todos/get.get
    events:
      - http:
          path: todos/{id}
          method: get
          cors: true

  update:
    handler: todos/update.update
    events:
      - http:
          path: todos/{id}
          method: put
          cors: true

  delete:
    handler: todos/delete.delete
    events:
      - http:
          path: todos/{id}
          method: delete
          cors: true
```

Once you have copied the todo Lambda functions into your project and updated the serverless.yml file we are ready to redeploy the project with the updated methods and test them out.

In [ ]:
!sls deploy -v

### Create a todo

In [ ]:
echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos
http https://{api-url}/dev/todos

### List the todos available

In [ ]:
!http POST https://{api-url}/dev/todos

Now you can see how easy it is to create a microservice using Lambda but what if we wanted to only allow authenticated users to have the ability to create todos? We can create a Cognito user pool and use it as an authorizer to the API. The deployment should have already created the Cognito user pool, so we will use that and hook it up to the create method of the service. 

Change the serverless.yml file to add the authorizer like below.

```
    authorizer:
        type: COGNITO_USER_POOLS
        authorizerId:
            Ref: TodoApiGatewayAuthorizer
```

Let's redeploy the service and try and add a todo

In [ ]:
!sls deploy -v

In [ ]:
!echo '{ "text": "My First Todo" }' | http POST https://{api-url}/dev/todos

This should have returned an HTTP/1.1 401 Unauthorized because you did not send the appropriate Authorization header on the call. We will simulate a user logging into the application and passing the bearer token in the headers of the request. Running the 3 methods below will simulate creating a user, confirming the user, and generating the appropriate auth for the user to be able to create the todo. Grab the IdToken generated and add it to the headers like the command below.

### Create user in Cognito

In [ ]:
!aws cognito-idp sign-up --region {your-aws-region} --client-id {cognito-client-id} --username admin@example.com --password {password}

### Confirm the sign up of the user

In [ ]:
!aws cognito-idp admin-confirm-sign-up --region {your-aws-region} --user-pool-id {cognito-user-pool} --username admin@example.com

### Initiate the authentication to generate the token

In [ ]:
!aws cognito-idp admin-initiate-auth --region {your-aws-region} --cli-input-json file://auth.json

In [ ]:
!echo '{ "text": "My Authenticated Todo" }' | http POST https://{api-url}/dev/todos Authorization:"Bearer {your-idtoken}"

Now you should have created a new todo after the authorizer approves the token.

### Register a user

> aws cognito-idp sign-up --region {your-aws-region} --client-id {your-client-id} --username admin@example.com --password {password}

### Confirm user registration

> aws cognito-idp admin-confirm-sign-up --region {your-aws-region} --user-pool-id {your-user-pool-id} --username admin@example.com

### Authenticate (get tokens)

> aws cognito-idp admin-initiate-auth --region {your-aws-region} --cli-input-json file://auth.json

### Where auth.json is:

>{
    "UserPoolId": "{your-user-pool-id}",
    "ClientId": "{your-client-id}",
    "AuthFlow": "ADMIN_NO_SRP_AUTH",
    "AuthParameters": {
        "USERNAME": "admin@example.com",
        "PASSWORD": "{password}"
    }
}

### You should get a response like this if everything is set up correctly:

>{
    "AuthenticationResult": {
        "ExpiresIn": 3600,
        "IdToken": "{your-idtoken}",
        "RefreshToken": "{your-refresh-token}",
        "TokenType": "Bearer",
        "AccessToken": "{your-access-token}"
    },
    "ChallengeParameters": {}
}

## Cleanup

In [86]:
!aws cloudformation delete-stack --stack-name $stack_name